In [1]:
subject = '''
ID: 1
Name: X
FamilyN: Y
Age: 20

ID: 2
Name: H
FamilyN: F
Age: 23

ID: 3
Name: S
FamilyN: Y
Age: 13

ID: 4
Name: M
FamilyN: Z
Age: 25'''

In [3]:
import re
result = re.findall(
    r"""(?mx)           # multiline, verbose regex
    ^ID:.*\s*           # Match ID: and anything else on that line 
    Name:\s*(.*)\s*     # Match name, capture all characters on this line
    FamilyN:\s*(.*)\s*  # etc. for family name
    Age:\s*(.*)$        # and age""", 
    subject)
print result

[('X', 'Y', '20'), ('H', 'F', '23'), ('S', 'Y', '13'), ('M', 'Z', '25')]


1. [extract numbers from mixed log file](http://stackoverflow.com/questions/32480483/extract-critical-numbers-from-a-mixed-log-file)


I have a log file contained many slices like this:

    Align set A and merge into set B ...
        setA, 4 images , image size 146 X 131
        setA, image 1, shape center shift (7, -9) compared to image center
        setA, image 2, shape center shift (8, -10) compared to image center
        setA, image 3, shape center shift (6, -9) compared to image center
        setA, image 4, shape center shift (6, -8) compared to image center
        final set B, image size 143 X 129
    Write set B ...

Now, I want to extract the numbers in this slice into a table:

| width_A | height_A | shift_x | shift_y | width_B | height_B|
--- | --- | --- | ----| ---
A1 | 146 | 131 | 7 | -9 | 143 | 129
A2 | 146 | 131 | 8 | -10 | 143 | 129
A3 | 146 | 131 | 6 | -9 | 143 | 129
A4 | 146 | 131 | 6 | -8 | 143 | 129

| width_A | height_A | shift_x1 | shift_y1 | shift_x2 | shift_y2 | shift_x3 | shift_y3 | shift_x4 | shift_y4 | width_B | height_B|
--- | --- | --- | ----| --- | --- | --- | ----| ---| --- | --- | ----|


If dividing the procedure into two parts, then:

1. text processing, read the text into a dictionary `data`, e.g., `data['A1']['shift_x'] = 7`. 
2. use pandas convert the dictionary into dataframe: `df = pd.DataFrame(data)`


But I am not familiar with python text processing:

 - Different from [Python: How to loop through blocks of lines](http://stackoverflow.com/questions/3914454/python-how-to-loop-through-blocks-of-lines), my log text are not so well organised; 
 - regular expression may be a choice, but I can never remember the tricks to classify all kinds of symbols
 
Does anyone have a good solution for this? Python is preferred. Thanks in advance.

#### most tricky part of this script is, ('gauge_no', ) & ('gauge_no') will outcome in different result, first is a turple, 2nd is a string

In [11]:
text = '''
gauge 1, BWall_01BW90N01920_01720_01320_00440_00960_00860_01320_00440_1_1, processing...
Merge set A into set B ...
    setA, 4 images , image size 321 X 291
    setA, image 1, shape center shift (15, -17) compared to image center
    setA, image 2, shape center shift (1, -17) compared to image center
    setA, image 3, shape center shift (13, -17) compared to image center
    setA, image 4, shape center shift (14, -17) compared to image center
    final set B, image size 319 X 263
Write gauge 1, set B ...
'''

# real example 2
import re
import os
import pandas as pd

# store attribute as a turple, construct a dictionary, turple_attribute: pattern
regexp = {
    ('gauge_no', ): re.compile('gauge (\d{1,}), BWall_.*' ),
    ('height_A', 'width_A'): re.compile('\s+setA, \d{1,} images , image size (\d{1,}) X (\d{1,}).*'),
    ('image_no', 'shift_x', 'shift_y'): re.compile('\s+setA, image (\d{1,}), shape center shift \((-?\d{1,}), (-?\d{1,})\) compared to image center.*'),
    ('height_B', 'width_B'): re.compile('\s+final set B, image size (\d{1,}) X (\d{1,})')} # ('gauge_no', ): re.compile(r'Write gauge (\d{1,}), set B.*')
    
#print(log_file)
dict_summary = {}
f = text.split('\n')
for line in f:   
    #print line
    for keys, pattern in regexp.iteritems():
        m = pattern.match(line)
        if m:          
            # traverse attributes
            for groupn, attr in enumerate(keys):  
                # print attr 
                if attr == 'gauge_no':
                    gauge_no = 'gauge' + str(m.group(groupn+1))
                    dict_summary[gauge_no] = {}
                elif attr == 'image_no':
                    image_no = m.group(groupn+1)
                elif (attr == 'shift_x') or (attr == 'shift_y'):
                    key = attr + str(image_no)
                    dict_summary[gauge_no][key] = int(m.group(groupn+1))
                else: # 'height_A', 'width_A', 'height_B', 'width_B'
                    dict_summary[gauge_no][attr] = int(m.group(groupn+1))
print dict_summary
df = pd.DataFrame(dict_summary)
df = df.transpose()
df['width_diff'] = df.apply(lambda x: x['width_A'] - x['width_B'], axis = 1)
df['height_diff'] = df.apply(lambda x: x['height_A'] - x['height_B'], axis = 1)
#df.to_csv(os.path.join(workpath, text_path, text_name+'_summary.csv'), sep = ',', na_rep='NaN')
print df.head(1)

{'gauge1': {'shift_x3': 13, 'shift_y1': -17, 'shift_y2': -17, 'shift_y3': -17, 'shift_y4': -17, 'height_A': 321, 'height_B': 319, 'shift_x2': 1, 'shift_x4': 14, 'width_B': 263, 'shift_x1': 15, 'width_A': 291}}
        height_A  height_B  shift_x1  shift_x2  shift_x3  shift_x4  shift_y1  \
gauge1       321       319        15         1        13        14       -17   

        shift_y2  shift_y3  shift_y4  width_A  width_B  width_diff  \
gauge1       -17       -17       -17      291      263          28   

        height_diff  
gauge1            2  


#### 2.  multiple matches

masksetting = 'PSM:defocus=0.06,PSM:dose=0,PSM:Delta_AI_location=0,binary:defocus=0,binary:dose=0,binary:Delta_AI_location=0'

In [7]:
import re
str1 = 'PSM:defocus=0.06,PSM:dose=0,PSM:Delta_AI_location=0,binary:defocus=0,binary:dose=0,binary:Delta_AI_location=0'
str2 = 'defocus=0,dose=0'
match0 = re.findall('(?P<{}>[a-zA-Z0-9_]+):(?P<{}>[a-zA-Z0-9_]+)=(?P<{}>[0-9.-]+)'.format( 
        'Exposure', 'key', 'val' ), str1 )
match2 = re.findall('(?P<{}>[a-zA-Z0-9_]+)=(?P<{}>[0-9.-]+)'.format( 
        'Exposure', 'key', 'val' ), str2 )
print "match0", match0
print "match2", match2
match1 = re.search('(?P<{}>[a-zA-Z0-9-]+):({})=(?P<{}>[0-9.-]+)'.format( 'Exposure', 'defocus', 'defocus'), str1 )
print 'match1: ',match1.group()+',',  match1.group(0)+',',  match1.group(1)+',',  match1.group(2)+',', match1.group(3)+','
print type(match2)
if not isinstance(match2, list):
    printREMatch(match2)
    def printREMatch(m):
        print len(m.group())
        print m.group()
        print m.group('defocus')
        print m.groups()
        print m.groupdict()

match0 [('PSM', 'defocus', '0.06'), ('PSM', 'dose', '0'), ('PSM', 'Delta_AI_location', '0'), ('binary', 'defocus', '0'), ('binary', 'dose', '0'), ('binary', 'Delta_AI_location', '0')]
match2 [('defocus', '0'), ('dose', '0')]
match1:  PSM:defocus=0.06, PSM:defocus=0.06, PSM, defocus, 0.06,
<type 'list'>


#### 3. job match

In [18]:
import re
import string
print string.punctuation
params = 'job0=/gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_05_PTD_Multiple_PRS_3_NTD_FT,job1=/gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc,job2=/gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc_FT,job3=/gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc_FT_NTD'
for m in re.findall(r'(\w+)=([^,=]+)', params): # [^:class:], negated class
    print "%s: %s" % (m[0].strip(), m[1].strip())
#jobpaths = re.findall(r'(?P<{}>[a-zA-Z0-9_/]+)'.format('alias'), params)
#print jobpaths

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
job0: /gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_05_PTD_Multiple_PRS_3_NTD_FT
job1: /gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc
job2: /gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc_FT
job3: /gpfs/PEG/FEM/SHARED/NTD_ModelDB/NTDCase56_SEC_10nm_New_tapeout_PRD/temp/Sook_test_TCC/Regular64_03_Optical_PRS_copy_Trunc_FT_NTD


In [8]:
import re
chars = '{Model Error} > percentile(75) + 1.5 * IQR or {Model Error} < percentile(25) - 1.5 * IQR'
m = re.findall(r'\{([^*/><+-]+)\}', chars)

['Model Error', 'Model Error']


NameError: name 'stdCol' is not defined

In [3]:
import re
xx = '+47'
m = re.findall(r'[+-]?\d+\.\d+|[+-]?\d+',xx)
print m

['+47']


In [14]:
import re
line ='512 512'
m = re.search(r'([0-9]+)\s([0-9]+)', line)            
imw=m.group(1)
imh=m.group(2)
print imw, imh

512 512


job sumbit server config xml match

In [3]:
import re
leafsevers = "leaf1, leaf2, leaf3"
leafsevers = '1,4'
a = re.findall(r'([^,\s\d]+)', leafsevers)
print a

[]


#### 4. FEM host log processing
#### most tricky part of this script is, ('gauge_no', ) & ('gauge_no') will outcome in different result, first is a turple, 2nd is a string

In [31]:
# -*- coding: utf-8 -*-
"""
log data processing
"""

# real example 2
import re
import os
import pandas as pd
hostlog = r'C:\Localdata\D\2Release\r1612E5.0\MOD7871_GA\Benchmark\lua_host.log'

# define 4 regexp formats: parmVals, chromosome N.O.,
regexp = {
    ('parmValsStart', 'stageIdx'): re.compile(r'<>genetic_algorithm_log, ([a-zA-Z]+), stageIdx\s*=\s*([0-9]+)'),
    ('chromNum', ): re.compile(r'\s*([0-9]+)\s*={' ),
    ('attr', ): re.compile(r'\s+([a-z]+)\s*={' ),
    ('parmName', 'parmVal', ): re.compile(r'\s+([\S]+)\s+=\s+([0-9.-]+)' ),

    ('rmsStart', ): re.compile(r'<>genetic_algorithm_log, ([a-zA-Z]+), rmses =\s*' ),
    ('chromNum', 'rms'): re.compile(r'\s+([0-9]+)\s+=\s+([0-9.]+)'),
    ('rmsEnd', ): re.compile(r'AppMain Don([\w]+)' ),
}

# read out all the log files
populations = {}
curAttr = 'NA'
columns = ['stageIdx', 'chromNum', 'rms', 'attr', 'parmName', 'parmVal']
df = pd.DataFrame()
with open(hostlog, 'r') as infile:
    # local variables initialization
    parmValsStart = False
    rmsStart = False
    curStageIdx = 0
    curChromNum = 0
    curAttr = 'NA'
    curParmName = 'NA'
    curSeries = pd.Series()
    curRMS = {}
    curdf = pd.DataFrame()

    for line in infile.readlines():
        #print line
        curParmName = 'NA'
        for keys, pattern in regexp.iteritems():
            m = pattern.match(line)
            if m:
                # traverse attributes
                for groupn, attr in enumerate(keys):
                    if attr == 'parmValsStart':
                        parmValsStart = True
                        curdf = pd.DataFrame()
                        curChromNum = 0
                    if attr == 'rmsStart':
                        parmValsStart = False
                        rmsStart = True
                        curRMS = {}
                    if attr == 'rmsEnd':
                        rmsStart = False
                        curdf['rms'] = curdf.apply(lambda x: curRMS[x['chromNum']], axis=1)
                        df = df.append(curdf)
                    if parmValsStart: # if-else order matters
                        if attr == 'stageIdx':
                            curStageIdx = int(m.group(groupn+1)) # match group content are whole group, val1, val2, ..
                        elif attr == 'chromNum':
                            curChromNum = int(m.group(groupn+1))
                            curAttr = 'NA'
                        elif attr == 'attr':
                            curAttr = m.group(groupn+1)
                        elif attr == 'parmName':
                            curParmName = m.group(groupn+1)
                        elif attr == 'parmVal':
                            curVal = m.group(groupn+1)
                            curSeries = pd.Series({'stageIdx': curStageIdx, 'chromNum': curChromNum,
                                                 'parmName': curParmName, 'parmVal': curVal, 'attr': curAttr})
                            curdf = curdf.append(curSeries, ignore_index=True)
                    if rmsStart:
                        if attr == 'chromNum':
                            curChromNum = int(m.group(groupn+1))
                        elif attr == 'rms':
                            rms = m.group(groupn+1)
                            curRMS[curChromNum] = rms

df = df[columns]
df = df.sort_values(by=['stageIdx', 'rms', 'attr', 'parmName'])
df.index = range(len(df))
print df.ix[range(0, len(df), 20), :]

     stageIdx  chromNum              rms    attr       parmName       parmVal
0           1         5   1.341043020549    mask    InnerCorner          0.02
20          1         5   1.341043020549  resist   sigmaCDO1_2d        0.0796
40          1         4  1.5960121768596  resist       sigmaAG2         0.063
60          1         3  1.7001165659423  resist  r3dShearRatio           0.2
80          1         2  1.9592673839835  optics      focusBlur          0.01
100         1         1  2.1811642842521    mask    InnerCorner          0.02
120         1         1  2.1811642842521  resist   sigmaCDO1_2d        0.0848
140         2         2  1.4315225186219  resist       sigmaAG2   0.057421875
160         2         4    1.55723335019  resist  r3dShearRatio           0.2
180         2         1   1.567419370604  optics      focusBlur          0.01
200         2         3  2.0044637717118    mask    InnerCorner          0.02
220         2         3  2.0044637717118  resist   sigmaCDO1_2d 

In [45]:
import re 

m = re.match(r'<>genetic_algorithm_log, ([a-zA-Z]+), jobparms\s*=\s*', '<>genetic_algorithm_log, AppStage, jobparms = $')
print m.group()+';', m.group(1)

m = re.match(r'stage GA_Main_Stage_([0-9]+) params:', 'stage GA_Main_Stage_1 params:')
print m.group()+';', m.group(1)

<>genetic_algorithm_log, AppStage, jobparms = ; AppStage
stage GA_Main_Stage_1 params:; 1


In [18]:
import re

sigmas = ['sigmaMGS', 'sigmaMGS1', 'sigma_bp1']
userterms = ['MGS', 'MGS1', 'Bp1']
# parmName = ['simgaMGS_shrk', 'sigmaMGS1_dev']
# sigmaParms = [aa for aa in parmNames if(re.match(r'sigma[\S]*', aa) and (not re.match(r'sigma2D[\S]*', aa)))]
# sigmaParms = [aa for aa in parmNames if(re.match(r'sigma[\S]*', aa) and (not re.match(r'sigma2D[\S]*', aa)))]
for term in userterms:
    for sigma in sigmas:
        if re.match(r'^sigma_?{}$'.format(term), sigma, re.I) or re.match(r'^sigma_?{}_[\S]+$'.format(term), sigma, re.I):
            print term, sigma

MGS sigmaMGS
MGS1 sigmaMGS1
Bp1 sigma_bp1


### Read the sem point files

In [22]:
import re
lines = ['SEM1 807',
'(12700024.243367,15310371.976502) 8.663847 34.717164 -8.631511 0.747836 1.000000 1.000000 12700023.548127 15310372.036738'
]

for line in lines:
    m = re.match(r'^(?P<sem>\w+)\s+(?P<npnts>\d+)$', line)
    if m is not None:
        semname = m.group('sem')
        npnts = m.group('npnts')
        idx = 0
        print semname, npnts
    else:
        idx += 1
#         mm = re.match(r'^\((?P<center_x>[0-9.-]+),\s*(?P<center_y>[0-9.-]+)\)', line)
        mm = re.match(r'^\([0-9.-]+,[0-9.-]+\)\s?[0-9.-]+\s?[0-9.-]+\s?[0-9.-]+\s?[0-9.-]+\s?[0-9.-]+\s?[0-9.-]+\s?(?P<center_x>[0-9.-]+)\s*(?P<center_y>[0-9.-]+)$', line)
#         print mm.group()
        print mm.group('center_x'), mm.group('center_y')
    


SEM1 807
12700023.548127 15310372.036738


### Parse Aligned SEM Spec

In [18]:
import string
from collections import OrderedDict
import re

def parseKW(s, outtersep=';', innersep=':', dtype=str):
    kws = s.split(outtersep)
    try:
        kws.remove('')
    except ValueError:
        pass
    kws = map(lambda x: map(string.strip, x.split(innersep)), kws)
    kws = map(lambda x: (x[0], dtype(x[1])), kws)
    return OrderedDict(kws)

if __name__ == '__main__':
    res = 'Aligned Spec: SUCCESS - COST:31.678214;PIXEL:1.000000;CENTER_X:-10475081.500000;CENTER_Y:-9505384.499977;THETA:0.000000\nAligned Spec: FAILS - COST:31.678214;PIXEL:1.000000;CENTER_X:-10475081.500000;CENTER_Y:-9505384.499977;THETA:0.000000'
    keys = ['COST', 'PIXEL', 'CENTER_X', 'CENTER_Y', 'THETA']
    
    success_pattern = re.compile('^Aligned Spec: (SUCCESS|FAILS) - ([\w\d+-.:;]+)')
    print('\t'.join(['SEM', 'STATUS'] + keys) + '\n')
    for line in res.strip().split('\n'):
        semix = '1'
        m = re.match(success_pattern, line)
        if m is not None:
            print m.groups()
            kws = parseKW(m.groups()[1])
            print kws
            print('\t'.join([semix, m.groups()[0]] + kws.values()) + '\n')


SEM	STATUS	COST	PIXEL	CENTER_X	CENTER_Y	THETA

('SUCCESS', 'COST:31.678214;PIXEL:1.000000;CENTER_X:-10475081.500000;CENTER_Y:-9505384.499977;THETA:0.000000')
OrderedDict([('COST', '31.678214'), ('PIXEL', '1.000000'), ('CENTER_X', '-10475081.500000'), ('CENTER_Y', '-9505384.499977'), ('THETA', '0.000000')])
1	SUCCESS	31.678214	1.000000	-10475081.500000	-9505384.499977	0.000000

('FAILS', 'COST:31.678214;PIXEL:1.000000;CENTER_X:-10475081.500000;CENTER_Y:-9505384.499977;THETA:0.000000')
OrderedDict([('COST', '31.678214'), ('PIXEL', '1.000000'), ('CENTER_X', '-10475081.500000'), ('CENTER_Y', '-9505384.499977'), ('THETA', '0.000000')])
1	FAILS	31.678214	1.000000	-10475081.500000	-9505384.499977	0.000000

